In [41]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# All game data
gameData = {
    'games': {}
}
   
# Loads a raw CSV file and creates a DF from it
def loadFile(filePath):
    inputDir = "./mens-machine-learning-competition-2018"
    return pd.read_csv(inputDir + "/" + filePath)

def getTeamId(teamName):
    return gameData['teams'][(gameData['teams']['TeamName'] == teamName)]['TeamID'].values[0]

def getTeamName(teamId):
    return gameData['teams'][(gameData['teams']['TeamID'] == teamId)]['TeamName'].values[0]

def getTeamWins(teamId):
    return gameData['annotatedDF'][(gameData['annotatedDF']['WTeamID'] == teamId)]

def getTeamLosses(teamId):
    return gameData['annotatedDF'][(gameData['annotatedDF']['LTeamID'] == teamId)]
    
def getGames(teamId):
    return teamWins(teamId).append(teamLosses(teamId))

def getGamesByYear(teamId, year):
    gamesWon = gameData['annotatedDF'][(gameData['annotatedDF']['Season'] == year) & (gameData['annotatedDF']['WTeamID'] == teamId)]
    gamesLost = gameData['annotatedDF'][(gameData['annotatedDF']['Season'] == year) & (gameData['annotatedDF']['LTeamID'] == teamId)]
    allGames = gamesWon.append(gamesLost)
    teams = gamesWon['LTeamID'].append(gamesLost['WTeamID'])
    games = {
        'wins': gamesWon,
        'losses': gamesLost,
        'full': allGames,
        'opponents': teams.unique()
    }
    return games

def haveStats(teamId, year):
    return (str(year) in gameData['games']) and (str(teamId) in gameData['games'][str(year)])

def getPointsPerPossession(teamData):
    offensive = teamData['wins'].loc[:,'WPPPOSS'].append(teamData['losses'].loc[:,'LPPPOSS'])
    defensive = teamData['wins'].loc[:,'LPPPOSS'].append(teamData['losses'].loc[:,'WPPPOSS'])
    retVal = {
        'offensive': {
            'ave': offensive.mean(),
            'std': offensive.std()
        },
        'defensive': {
            'ave': defensive.mean(),
            'std': defensive.std()
           
        }
    }
    return retVal

#  Go through the dataframe and add possession information to the box score
def addData(df):
    augmented = df.assign(
        WFGPERC = df.WFGM / df.WFGA, 
        WFGPERC3 = df.WFGM3 / df.WFGA3,
        LFGPERC = df.LFGM / df.LFGA,
        LFGPERC3 = df.LFGM3 / df.LFGA3,
        WORPERC = df.WOR / (df.WFGA + df.WFGA3),
        LORPERC = df.LOR / (df.LFGA + df.LFGA3),
        WPOSS = df.WFGA + df.WFGA3 + df.LTO,
        LPOSS = df.LFGA + df.LFGA3 + df.WTO,
        MOV = df.WScore - df.LScore
    )
    second = augmented.assign(
        WPPPOSS = augmented.WScore / augmented.WPOSS,
        LPPPOSS = augmented.LScore / augmented.LPOSS,
        MOP = augmented.WPOSS - augmented.LPOSS
    )

    finalFrame = second.assign(
        MOPPPOSS = second.WPPPOSS - second.LPPPOSS
    )
    
    return finalFrame

# Add team to the given year
def addTeam(teamId,year):
    if not str(year) in gameData['games']:
        gameData['games'][str(year)] = {}
    gameData['games'][str(year)][str(teamId)] = {}

# Add the team and add the team's games and opponents for the given year
def addAllGames(team,year):
    teamId=getTeamId(team)
    if not haveStats(teamId, year):
        addTeam(teamId,year)
        team = getTeamStats(teamId, year)
        teamGames = getGamesByYear(teamId, year)
        team['games'] = teamGames
        addAllOpponents(teamId,year)

# Add all the opponents for a team for the given year
def addAllOpponents(teamId,year):
    for oppId in getTeamStats(teamId, year)['games']['opponents']:
        addAllGames(getTeamName(oppId),year)

# Get the stats for the given team/year
def getTeamStats(teamId,year):
    return gameData['games'][str(year)][str(teamId)]

# Return games between two teams in a year        
def findGamesBetweenTeams(team1, team2, year):
    team1Id = getTeamId(team1)
    team2Id = getTeamId(team2)
    return gameData['annotatedDF'][(gameData['annotatedDF']['Season'] == year) & (
        (gameData['annotatedDF']['WTeamID'] == team1Id) & (gameData['annotatedDF']['LTeamID'] == team2Id) | 
        (gameData['annotatedDF']['WTeamID'] == team2Id) & (gameData['annotatedDF']['LTeamID'] == team1Id))
        ]

# ******************************
# Display functions
# ******************************
def displayOpponents(teamId, year):
    for opponentId in getTeamStats(teamId, year)['games']['opponents']:
        displayStats(opponentId, year)

def displayStats(teamId, year):
    if haveStats(teamId, year):
        teamName = getTeamName(teamId)
        teamStats = getTeamStats(teamId, year)
        print(teamStats)
    else:
        print('No status for: ' + getTeamName(teamId))
    
# ******************************
# Main
# ******************************
seasonsFile = "Seasons.csv"
playersFile = "Players_2018.csv"
eventsFile = "Events_2018.csv"
resultsFile = "RegularSeasonCompactResults.csv"
teamsFile= 'Teams.csv'

# Team name/ids map
gameData['teams'] = loadFile(teamsFile)

# Annotate box scores
gameData['annotatedDF'] = addData(loadFile('RegularSeasonDetailedResults.csv'))


In [84]:
def displayGameStats(team, year):
    teamId = getTeamId(team)
    if(haveStats(teamId, year)):
        annotated = gameData['annotatedDF'][(gameData['annotatedDF']['Season'] == year)]
        season = annotated[(annotated['Season'] == year)]
        wins = season[(season['WTeamID'] == teamId)]
        losses = season[(season['LTeamID'] == teamId)]
        allOpps = wins.loc[:,'LTeamID'].append(losses.loc[:,'WTeamID']).unique()
        data = {}
        data['opponents'] = {}
        for oppTeam in allOpps:
            # Get all of the opponent's games that weren't against our team
            oppName = getTeamName(oppTeam)
            data['opponents'][oppName] = {}
            
            # Opponent's stats against their opponents
            oppWins = season[(season['WTeamID'] == oppTeam) & (season['LTeamID'] != teamId)]
            oppLoss = season[(season['LTeamID'] == oppTeam) & (season['WTeamID'] != teamId)]
            oppPoss = oppWins.loc[:, 'WPOSS'].append(oppLoss.loc[:, 'LPOSS'])
            oppPossAve = oppPoss.mean()
            oppPossStd = oppPoss.std()
            
            # Opponent's stats vs us
            gamesVsWins = season[(season['WTeamID'] == oppTeam) & (season['LTeamID'] == teamId)]
            gamesVsLoss = season[(season['LTeamID'] == oppTeam) & (season['WTeamID'] == teamId)]
            gameVsPoss = gamesVsWins.loc[:, 'WPOSS'].append(gamesVsLoss.loc[:, 'LPOSS'])
            gameVsPossAve = gameVsPoss.mean()
            
            # Our stats against all other teams not this one
            teamWins = season[(season['WTeamID'] == teamId) & (season['LTeamID'] != oppTeam)]
            teamLoss = season[(season['LTeamID'] == teamId) & (season['WTeamID'] != oppTeam)]
            teamPoss = teamWins.loc[:, 'WPOSS'].append(teamLoss.loc[:, 'LPOSS'])
            teamPossAve = teamPoss.mean()
            teamPossStd = teamPoss.std()
            
            # our stats against this team
            teamVsWins = season[(season['WTeamID'] == teamId) & (season['LTeamID'] == oppTeam)]
            teamVsLoss = season[(season['LTeamID'] == teamId) & (season['WTeamID'] == oppTeam)]
            teamVsPoss = teamVsWins.loc[:, 'WPOSS'].append(teamVsLoss.loc[:, 'LPOSS'])
            gameVsPossAve = teamVsPoss.mean()
            
            print('vs ' + oppName)
            print('oppPossAve: ' + str(oppPossAve))
            print('oppPossStd: ' + str(oppPossStd))
            print('gameVsPossAve: ' + str(gameVsPossAve))
            print('teamPossAve: ' + str(teamPossAve))
            print('teamPossStd: ' + str(teamPossStd))
    else:
        print('No status for: ' + getTeamName(teamId))
        

addAllGames('Utah',2017)
displayGameStats('Utah', 2017)

#foo = findGamesBetweenTeams('Utah', 'Colorado', 2017)
#print(foo)

#addAllGames(getTeamId('Utah'), 2017)
#displayStats(getTeamId('Utah'),2017)
# displayStats(getTeamId('Utah'),2016)

# displayStats(getTeamId('UCLA'),2017)
# displayStats(getTeamId('Arizona'),2017)
# displayStats(getTeamId('USC'),2017)
# displayStats(getTeamId('North Carolina'),2017)


vs Coppin St
oppPossAve: 92.89655172413794
oppPossStd: 13.859872261795939
gameVsPossAve: 105.0
teamPossAve: 87.67857142857143
teamPossStd: 7.159834321632556
vs UC Riverside
oppPossAve: 87.8076923076923
oppPossStd: 9.883397111395377
gameVsPossAve: 99.0
teamPossAve: 87.89285714285714
teamPossStd: 7.588193976875383
vs Montana St
oppPossAve: 101.55172413793103
oppPossStd: 14.833808409211112
gameVsPossAve: 96.0
teamPossAve: 88.0
teamPossStd: 7.726817178094982
vs Utah Valley
oppPossAve: 99.37037037037037
oppPossStd: 11.348155214863198
gameVsPossAve: 95.0
teamPossAve: 88.03571428571429
teamPossStd: 7.762598566713116
vs Prairie View
oppPossAve: 91.16666666666667
oppPossStd: 10.252277571461809
gameVsPossAve: 76.0
teamPossAve: 88.71428571428571
teamPossStd: 7.497442244635181
vs Hawaii
oppPossAve: 90.21428571428571
oppPossStd: 7.156230866289894
gameVsPossAve: 79.0
teamPossAve: 88.60714285714286
teamPossStd: 7.661057299211313
vs SF Austin
oppPossAve: 87.85714285714286
oppPossStd: 10.93656553075350